# CNM_CycNucMed — Clean Colab/Jupyter Runner (run_pipeline)

This notebook is minimal and avoids argparse issues by calling the **library** entrypoint:

```python
from iaea_project.pipeline import run_pipeline
```

If you edited `pipeline.py`, re-run the install cell (or restart the runtime) so Colab picks up changes.


In [1]:
# =========================
# 1) Clone / pull + cd
# =========================
import os, subprocess, pathlib

REPO_URL = "https://github.com/tommasocarzaniga/CNM_CycNucMed.git"
REPO_DIR = "CNM_CycNucMed"

def sh(cmd, cwd=None):
    print(">>", cmd)
    subprocess.check_call(cmd, shell=True, cwd=cwd)

# If you get an auth error, your repo is private.
# Clone using a GitHub Personal Access Token (PAT):
#   from getpass import getpass
#   token = getpass("GitHub PAT (will not be shown): ").strip()
#   sh(f"git clone https://{token}@github.com/tommasocarzaniga/CNM_CycNucMed.git {REPO_DIR}")

if not os.path.exists(REPO_DIR):
    sh(f"git clone {REPO_URL}")
else:
    sh("git pull", cwd=REPO_DIR)

%cd {REPO_DIR}
print("Now in:", pathlib.Path().resolve())


>> git pull
/content/CNM_CycNucMed
Now in: /content/CNM_CycNucMed


In [2]:
# =========================
# 2) Install deps + package (editable)
# =========================
import sys, subprocess, pathlib

def pip(cmd: str):
    print(">> pip", cmd)
    subprocess.check_call([sys.executable, "-m", "pip"] + cmd.split())

pip("install -U pip setuptools wheel")

root = pathlib.Path('.').resolve()
if (root / 'requirements.txt').exists():
    pip(f"install -r {root/'requirements.txt'}")

# Editable install (pyproject.toml / src layout supported)
pip("install -e .")


>> pip install -U pip setuptools wheel
>> pip install -r /content/CNM_CycNucMed/requirements.txt
>> pip install -e .


In [3]:
# =========================
# 3) Verify pipeline entrypoint (avoid 'main'/argparse)
# =========================
import importlib
import iaea_project.pipeline as pipeline

# Force reload in case you edited files after the first import
pipeline = importlib.reload(pipeline)

print("pipeline module:", pipeline.__file__)
print("Has run_pipeline:", hasattr(pipeline, "run_pipeline"))
assert hasattr(pipeline, "run_pipeline"), (
    "run_pipeline() not found. Make sure src/iaea_project/pipeline.py defines run_pipeline and that you re-ran the install cell."
)


pipeline module: /content/CNM_CycNucMed/src/iaea_project/pipeline.py
Has run_pipeline: True


In [4]:
# =========================
# Dependencies (Colab)
# =========================
#!pip -q install playwright

# Install Chromium + system deps in one go (recommended)
!playwright install --with-deps chromium



Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-freefont-ttf is already the newest version (20120503-10build1

**Running the Pipeline via Python API**

This module allows you to execute the IAEA cyclotron reporting pipeline programmatically from Python.
The run_pipeline function runs the full data processing workflow and returns the file path of the generated PDF report.

What it does:
*   Executes the data pipeline (data loading, processing, summarization, and report generation)
*   Supports running for all countries or a selected subset
*   Returns the absolute path to the generated PDF file

In [5]:
from iaea_project.pipeline import run_pipeline

# Run all countries:
# pdf_path = run_pipeline()

# Run selected countries:
pdf_path = run_pipeline(["Switzerland"])  # edit as needed

print("Generated PDF at:", pdf_path)


Saved raw:   /content/CNM_CycNucMed/data/raw/iaea_cyclotrons_raw.csv
Saved clean: /content/CNM_CycNucMed/data/processed/iaea_cyclotrons_clean.csv
Saved PDF:   /content/CNM_CycNucMed/outputs/reports/IAEA_Cyclotron_Report.pdf
Generated PDF at: /content/CNM_CycNucMed/outputs/reports/IAEA_Cyclotron_Report.pdf


**cyclotron_run — CLI Wrapper**

*cyclotron_run* is a lightweight command-line interface (CLI) wrapper designed to simplify the execution of cyclotron-related data pipelines and analysis workflows.
It provides a clean, consistent entry point to run complex scripts with clear commands and arguments, making the tool easier to use, automate, and reproduce.

Instead of manually editing scripts or remembering long commands, users can interact with the system through intuitive commands such as:

*   cyclotron_run country --name "Switzerland"
*   cyclotron_run full

The goal of cyclotron_run is to improve:
*   Reproducibility of analyses
*   Usability for non-developers
*   Maintainability of the codebase
*   Scalability as new features and modules are added

In [9]:
#!python scripts/cyclotron_run.py --countries Switzerland Germany
